In [ ]:
import copy
import torch
import numpy as np
import pandas as pd
import functions as fn
from torch.utils.data import DataLoader
from tqdm import tqdm
import models
import learner
import time
import GAT

In [ ]:
# system configuration
use_cuda = True
device = torch.device("cuda:0" if use_cuda and torch.cuda.is_available() else "cpu")
fn.set_seed(seed=2023, flag=True)

torch.cuda.empty_cache()

In [ ]:
# hyper params
model_name = 'PAG'
seq_l = 12
pre_l = 6
node_l = 247
bs = 512
p_epoch = 200
n_epoch = 1000
law_list = np.array([-1.48, -0.74])  # price elasticities of demand for EV charging. Recommend: up to 5 elements.
is_train = True
mode = 'completed'  # 'simplified' or 'completed'
is_pre_train = True

In [ ]:
# input data
occ, prc, adj, col, dis, cap, tim, inf = fn.read_dataset()
adj_dense = torch.Tensor(adj)
adj_dense_cuda = adj_dense.to(device)
adj_sparse = adj_dense.to_sparse_coo().to(device)

In [ ]:
# dataset division
train_occupancy, valid_occupancy, test_occupancy = fn.division(occ, train_rate=0.6, valid_rate=0.2, test_rate=0.2)
nodes = train_occupancy.shape[-1]
train_price, valid_price, test_price = fn.division(prc, train_rate=0.6, valid_rate=0.2, test_rate=0.2)


In [ ]:
# data
train_dataset = fn.CreateDataset(train_occupancy, train_price, seq_l, pre_l, device, adj_dense)
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, drop_last=True)
valid_dataset = fn.CreateDataset(valid_occupancy, valid_price, seq_l, pre_l, device, adj_dense)
valid_loader = DataLoader(valid_dataset, batch_size=len(valid_occupancy), shuffle=False)
test_dataset = fn.CreateDataset(test_occupancy, test_price, seq_l, pre_l, device, adj_dense)
test_loader = DataLoader(test_dataset, batch_size=len(test_occupancy), shuffle=False)


In [ ]:
# training setting
model = GAT.GAT_Multi(seq_l, 2, 1, 0, 0.2, 1).to(device)
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.00001)

loss_function = torch.nn.MSELoss()
valid_loss = 100

In [ ]:
if is_train is True:
    model.train()
    if is_pre_train is True:
        if mode == 'simplified':  # a simplified way of physics-informed meta-learning
            model = learner.fast_learning(law_list, model, model_name, p_epoch, bs, train_occupancy, train_price, seq_l, pre_l, device, adj_dense)

        elif mode == 'completed': # the completed process
            model = learner.physics_informed_meta_learning(law_list, model, model_name, p_epoch, bs, train_occupancy, train_price, seq_l, pre_l, device, adj_dense)
        else:
            print("Mode error, skip the pre-training process.")

    for epoch in tqdm(range(n_epoch), desc='Fine-tuning'):
        for j, data in enumerate(train_loader):
            '''
            occupancy = (batch, seq, node)
            price = (batch, seq, node)
            label = (batch, node)
            '''
            model.train()
            occupancy, price, label = data

            optimizer.zero_grad()
            predict = model(occupancy, adj_dense_cuda)
            loss = loss_function(predict, label)
            loss.backward()
            optimizer.step()

        # validation
        model.eval()
        for j, data in enumerate(valid_loader):
            '''
            occupancy = (batch, seq, node)
            price = (batch, seq, node)
            label = (batch, node)
            '''
            model.train()
            occupancy, price, label = data
            predict = model(occupancy, adj_dense_cuda)
            loss = loss_function(predict, label)
            if loss.item() < valid_loss:
                valid_loss = loss.item()
                torch.save(model, './checkpoints' + '/' + model_name + '_' + str(pre_l) + '_bs' + str(bs) + '_' + mode + '.pt')

In [ ]:
model = torch.load('./checkpoints' + '/' + model_name + '_' + str(pre_l) + '_bs' + str(bs) + '_' + mode + '.pt')
# test
model.eval()
result_list = []
predict_list = np.zeros([1, adj_dense.shape[1]])
label_list = np.zeros([1, adj_dense.shape[1]])

In [ ]:
# Initialize lists to store time and memory usage
time_list = []
memory_list = []

In [ ]:
for j, data in enumerate(test_loader):
    occupancy, price, label = data  # occupancy.shape = [batch, seq, node]
    print('occupancy:', occupancy.shape, 'price:', price.shape, 'label:', label.shape)
    with torch.no_grad():
        # Start time measurement
        start_time = time.time()
        
        # Start memory tracking
        if torch.cuda.is_available():
            torch.cuda.synchronize()  # Synchronize before measuring
            memory_before = torch.cuda.memory_allocated()

        predict = model(occupancy, adj_dense_cuda)

        # End memory tracking
        if torch.cuda.is_available():
            memory_after = torch.cuda.memory_allocated()
            memory_usage = memory_after - memory_before
            memory_list.append(memory_usage / (1024 * 1024))  # Convert bytes to MB
        
        # End time measurement
        end_time = time.time()
        elapsed_time = end_time - start_time
        time_list.append(elapsed_time)

        predict = predict.cpu().detach().numpy()
        label = label.cpu().detach().numpy()
        predict_list = np.concatenate((predict_list, predict), axis=0)
        label_list = np.concatenate((label_list, label), axis=0)


In [ ]:
output_no_noise = fn.metrics(test_pre=predict_list[1:, :], test_real=label_list[1:, :])
result_list.append(output_no_noise)
result_df = pd.DataFrame(columns=['MSE', 'RMSE', 'MAPE', 'RAE', 'MAE', 'R2'], data=result_list)
result_df.to_csv('./results' + '/' + model_name + '_' + str(pre_l) + 'bs' + str(bs) + '.csv', encoding='gbk')


In [ ]:
# Print average time and memory usage
print(f'Average time per prediction: {np.mean(time_list)} seconds')
print(f'Average memory usage per prediction: {np.mean(memory_list)} MB')

In [ ]:
import torch
import torch.nn as nn
import models
import torch.nn.functional as F
import functions as fn
import copy

from torch.nn import Transformer, TransformerEncoder, TransformerEncoderLayer


In [ ]:
class GraphAttentionFuncLayer(nn.Module):

    def __init__(self, in_features, out_features, node_l, dropout, alpha, concat=True):
        super(GraphAttentionFuncLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat
        self.node_l = node_l

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.kaiming_normal_(self.W, mode='fan_out', nonlinearity='leaky_relu')
        self.a = nn.Parameter(torch.empty(2*out_features, 1))
        nn.init.xavier_normal_(self.a, gain=nn.init.calculate_gain('leaky_relu', param=alpha))
        self.time_embedding = nn.Parameter(torch.empty(in_features, node_l))
        nn.init.xavier_normal_(self.time_embedding, gain=nn.init.calculate_gain('leaky_relu', param=alpha))
        self.node_embedding = nn.Parameter(torch.empty(node_l, in_features))
        nn.init.xavier_normal_(self.node_embedding, gain=nn.init.calculate_gain('leaky_relu', param=alpha))

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, input, adj):
        batch_size, N, _ = input.size() # x = [input, N, in_features]
        if adj.dim() == 3:
            adj = adj[:,:,1].unsqueeze(2).repeat(1,1,adj.shape[1])  # 扩展为 [batch_size, N, N]
        elif adj.size(0) != batch_size:
            adj = adj[:,:].unsqueeze(0).repeat(batch_size, 1, 1)

        '''
        h = torch.matmul(input, self.W)  # [batch_size, N, out_features]

        h_repeated1 = h.unsqueeze(2).expand(-1, -1, N, -1)  # [batch_size, N, N, out_features]
        h_repeated2 = h.unsqueeze(1).expand(-1, N, -1, -1)  # [batch_size, N, N, out_features]
        a_input = torch.cat([h_repeated1, h_repeated2], dim=-1)  # [batch_size, N, N, 2*out_features]

        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(-1))  # [batch_size, N, N]"
        '''

        time_embeddings = self.leakyrelu(torch.matmul(input, self.time_embedding)) # [batch_size, N, N]
        embeddings = self.leakyrelu(torch.matmul(time_embeddings, self.node_embedding)) # [batch_size, N, in_features]

        h = torch.matmul(embeddings, self.W)  # [batch_size, N, out_features]

        h_repeated1 = h.unsqueeze(2).expand(-1, -1, N, -1)  # [batch_size, N, N, out_features]
        h_repeated2 = h.unsqueeze(1).expand(-1, N, -1, -1)  # [batch_size, N, N, out_features]
        a_input = torch.cat([h_repeated1, h_repeated2], dim=-1)  # [batch_size, N, N, 2*out_features]

        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(-1))  # [batch_size, N, N]"

        if adj.dim() == 2:
            adj = adj.unsqueeze(0).expand(batch_size, -1, -1)  # 扩展为 [batch_size, N, N]

        zero_vec = -9e15 * torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)  # [batch_size, N, N]

        attention = F.softmax(attention, dim=-1)
        attention = F.dropout(attention, self.dropout, training=self.training)

        h_prime = torch.matmul(attention, h)  # [batch_size, N, out_features]

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

In [ ]:
GAF_model = GraphAttentionFuncLayer(seq_l, 1, node_l, 0.2, 0.2).to(device)

In [ ]:
GAF_model.eval()

In [ ]:
GAF_predict = GAF_model(occupancy, adj_dense_cuda)

In [ ]:
GAF_predict